In [1]:
from datetime import datetime, date
import logging
import yfinance as yf
from dotenv import load_dotenv
import dataUtil as DU
import json
from os import environ
import pandas as pd
import sys
import pytz

logger = logging.getLogger()
logger.setLevel(logging.INFO)
load_dotenv() 


C:\Users\Asus\anaconda3\envs\QuantAnalysis\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


True

In [2]:
def getOptions(ticker, PnC, strike, expiration):
    exp_dt = datetime.strptime(expiration, "%Y-%m-%d")
    today = date.today()
    logging.info(f'-->getOptions({ticker},{PnC},{strike},{expiration},{exp_dt},{today})')
    op_price = None
    pclose = -0.001
    if (today > exp_dt.date()):
        logging.error(f'  expiration: {exp_dt} is old\n')
        return pclose, op_price
    
    asset = yf.Ticker(ticker)
    
    histdata = asset.history()
    if len(histdata)>0:
        # logging.info(f'asset.history:\n {histdata}')
        pclose = histdata.iloc[-1].Close
        pclose = float("{:.2f}".format(pclose))
        ex_dates = asset.options
        logging.info(f'  {ticker}.Options expiration dates is {ex_dates}')
        try:
            opts = asset.option_chain(expiration)
            logging.info(f' opts found:\n {opts}')
            if PnC == 'P':
                op = opts.puts[opts.puts['strike'] == strike]
                # print(op)
            else:
                op = opts.calls[opts.calls['strike'] == strike]
                # print(op)
            if len(op)>0:
                opt = op.head(1).reset_index()
                op_price = op.iloc[0]
            else:
                logging.error(f'{ticker} {strike}{PnC} not found\n')
        except Exception as error:
            logging.error(error)
    return pclose, op_price


In [3]:
pclo, op_p = getOptions('HYG', 'P', 74, '2024-06-28')
print(pclo, "\n", op_p)

INFO:root:-->getOptions(HYG,P,74,2024-06-28,2024-06-28 00:00:00,2024-06-06)
INFO:root:  HYG.Options expiration dates is ('2024-06-07', '2024-06-14', '2024-06-21', '2024-06-28', '2024-07-05', '2024-07-12', '2024-07-19', '2024-08-16', '2024-09-20', '2024-10-18', '2024-11-15', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-05-16', '2025-06-20', '2026-01-16')
INFO:root: opts found:
 Options(calls=       contractSymbol             lastTradeDate  strike  lastPrice   bid  \
0  HYG240628C00076000 2024-05-31 14:01:05+00:00    76.0       1.05  1.07   
1  HYG240628C00076500 2024-06-03 15:04:15+00:00    76.5       0.85  0.70   
2  HYG240628C00077000 2024-05-30 19:30:02+00:00    77.0       0.24  0.31   
3  HYG240628C00077500 2024-06-05 14:38:20+00:00    77.5       0.33  0.00   
4  HYG240628C00078000 2024-06-06 14:05:26+00:00    78.0       0.03  0.02   
5  HYG240628C00078500 2024-05-16 13:40:17+00:00    78.5       0.40  0.00   
6  HYG240628C00079000 2024-06-03 14:50:04+0

77.15 
 contractSymbol              HYG240628P00074000
lastTradeDate        2024-05-31 19:27:21+00:00
strike                                    74.0
lastPrice                                 0.06
bid                                        0.0
ask                                       0.49
change                                     0.0
percentChange                              0.0
volume                                  1500.0
openInterest                              1500
impliedVolatility                     0.210701
inTheMoney                               False
contractSize                           REGULAR
currency                                   USD
Name: 5, dtype: object


In [8]:
pclo, op_p = getOptions('LQD', 'P', 103, '2024-06-28')
print(pclo, ", ", op_p['bid'], "\n", op_p)

INFO:root:-->getOptions(LQD,P,103,2024-06-28,2024-06-28 00:00:00,2024-06-06)
INFO:root:  LQD.Options expiration dates is ('2024-06-07', '2024-06-14', '2024-06-21', '2024-06-28', '2024-07-05', '2024-07-12', '2024-07-19', '2024-08-16', '2024-09-20', '2024-10-18', '2024-11-15', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-05-16', '2026-01-16')
INFO:root: opts found:
 Options(calls=        contractSymbol             lastTradeDate  strike  lastPrice   bid  \
0   LQD240628C00104000 2024-05-24 18:30:32+00:00   104.0       3.15  4.20   
1   LQD240628C00105000 2024-06-05 19:18:39+00:00   105.0       3.39  2.92   
2   LQD240628C00105500 2024-05-31 19:56:16+00:00   105.5       1.75  2.84   
3   LQD240628C00106000 2024-05-31 19:56:16+00:00   106.0       1.38  2.39   
4   LQD240628C00106500 2024-05-31 19:56:16+00:00   106.5       1.02  1.96   
5   LQD240628C00107000 2024-06-04 13:30:19+00:00   107.0       1.29  1.57   
6   LQD240628C00107500 2024-06-05 18:08:51+00:00 

107.92 ,  0.05 
 contractSymbol              LQD240628P00103000
lastTradeDate        2024-06-03 13:30:04+00:00
strike                                   103.0
lastPrice                                 0.38
bid                                       0.05
ask                                       0.15
change                                     0.0
percentChange                              0.0
volume                                     5.0
openInterest                                 6
impliedVolatility                     0.137704
inTheMoney                               False
contractSize                           REGULAR
currency                                   USD
Name: 9, dtype: object


In [6]:
pclo, op_p = getOptions('CSCO', 'C', 50, '2024-06-21')
print(pclo, "\n", op_p)

INFO:root:-->getOptions(CSCO,C,50,2024-06-21,2024-06-21 00:00:00,2024-06-06)
INFO:root:  CSCO.Options expiration dates is ('2024-06-07', '2024-06-14', '2024-06-21', '2024-06-28', '2024-07-05', '2024-07-12', '2024-07-19', '2024-08-16', '2024-09-20', '2024-10-18', '2025-01-17', '2025-03-21', '2025-06-20', '2025-09-19', '2025-12-19', '2026-01-16')
INFO:root: opts found:
 Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0   CSCO240621C00020000 2024-05-29 13:37:02+00:00    20.0      26.10  26.25   
1   CSCO240621C00022500 2023-01-04 14:30:13+00:00    22.5      26.18  26.35   
2   CSCO240621C00025000 2024-04-03 14:09:24+00:00    25.0      24.35  22.30   
3   CSCO240621C00027500 2024-04-10 17:02:04+00:00    27.5      21.80  18.70   
4   CSCO240621C00030000 2024-05-31 19:55:00+00:00    30.0      16.60  16.25   
5   CSCO240621C00032500 2024-06-04 16:39:21+00:00    32.5      14.60  13.65   
6   CSCO240621C00035000 2024-05-22 19:01:23+00:00    35.0     

46.23 
 contractSymbol             CSCO240621C00050000
lastTradeDate        2024-06-06 15:41:53+00:00
strike                                    50.0
lastPrice                                 0.03
bid                                       0.02
ask                                       0.03
change                                     0.0
percentChange                              0.0
volume                                    77.0
openInterest                             44178
impliedVolatility                     0.207039
inTheMoney                               False
contractSize                           REGULAR
currency                                   USD
Name: 22, dtype: object
